In [5]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten,BatchNormalization

from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
seed = 16
filename = 'fer2013.csv'
label_map = ['Enfado', 'Asco', 'Miedo', 'Alegria', 'Tristeza', 'Sorpresa', 'Neutral']
names=['emocion','pixels','uso']
df=pd.read_csv('fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emocion,pixels,uso
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [4]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        #This condition skips the first condition
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X)/255.0 , np.array(Y)
    return X,Y

In [5]:
X,Y = getData("fer2013.csv")

In [6]:
N,D = X.shape
X=X.reshape(N, 48, 48, 1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=16 )

# one hot encode output
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [61]:
model = Sequential()


In [62]:
model.add(Conv2D(48,(2,2),input_shape=X_train.shape[1:],padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


In [63]:
model.add(Conv2D(24,(2,2),input_shape=X_train.shape[1:],padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [64]:
model.add(Flatten())

In [65]:
model.add(Dense(300, kernel_constraint=maxnorm(2)))

In [66]:
model.add(Activation('relu'))
model.add(Dense(7, kernel_constraint=maxnorm(2)))
model.add(Activation('relu'))

In [67]:
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 48, 48, 48)        240       
_________________________________________________________________
activation_20 (Activation)   (None, 48, 48, 48)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 24, 24, 48)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 24)        4632      
_________________________________________________________________
activation_21 (Activation)   (None, 24, 24, 24)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 24)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3456)             

In [68]:
epochs = 20
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [69]:
np.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Train on 28709 samples, validate on 7178 samples
Epoch 1/20
28709/28709 [==============================] - 41s 1ms/step - loss: 5.6476 - accuracy: 0.2127 - val_loss: 3.4192 - val_accuracy: 0.2469
Epoch 2/20
28709/28709 [==============================] - 42s 1ms/step - loss: 3.3379 - accuracy: 0.2724 - val_loss: 3.2813 - val_accuracy: 0.2997
Epoch 3/20
28709/28709 [==============================] - 42s 1ms/step - loss: 3.3253 - accuracy: 0.3003 - val_loss: 3.2964 - val_accuracy: 0.3373
Epoch 4/20
28709/28709 [==============================] - 44s 2ms/step - loss: 3.2349 - accuracy: 0.3400 - val_loss: 3.1984 - val_accuracy: 0.3129
Epoch 5/20
28709/28709 [==============================] - 46s 2ms/step - loss: 3.2109 - accuracy: 0.3180 - val_loss: 3.2633 - val_accuracy: 0.2467
Epoch 6/20
28709/28709 [==============================] - 46s 2ms/step - loss: 3.2301 - accuracy: 0.2813 - val_loss: 3.2310 - val_accuracy: 0.2948
Epoch 7/20
28709/28709 [==============================] - 45s 2ms/ste

In [70]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 36.11%
